In [1]:
import math
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import os
import pickle
import matplotlib 
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import KFold

%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import imblearn as imb

from sklearn.metrics import confusion_matrix
import itertools

from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix, matthews_corrcoef,balanced_accuracy_score

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

import sys
sys.path.append('../../')
# import utils
from sklearn.model_selection import GridSearchCV

from split_data import *
# from RF_Utils import *
# from RF_GSCV import *
from RF_atomver import *

In [2]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
nek_nums = [2,3,5,9]
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
features = ['moe', 'mfp']
train_ds_sizes = []
train_actives = [] 
test_ds_sizes = []
test_actives = [] 
ds_names = []
neks_label = []
samp_types = []
feat_types = [] 
count=0
neks = []
for i, n in enumerate(nek_nums):
    nek = str(n)
    model_type = 'RF'
    nek_path= f'{data_dir}NEK{nek}/bind/'
 
    for k, feat in enumerate(features): 
        print()
        for j, samp in enumerate(samplings): 
            count+=1
            print(f'{count}. NEK{nek} bind {feat} {samp} {model_type}')
            file_root = f'NEK{nek}_binding_{feat}_{samp}_df.csv'
            neks.append(f'NEK{nek}_binding')
            trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
            train_ds_sizes.append(trainX.shape[0]) 
            train_actives.append(np.sum(trainy==1))
            test_ds_sizes.append(testX.shape[0]) 
            test_actives.append(np.sum(testy==1))
            ds_names.append(file_root)
            samp_types.append(samp)
            neks_label.append(f'NEK{nek}_binding')
            feat_types.append(feat)
            
            
        
            # print()
          
            
    if n == 2 or n == 9:

        nek_path= f'{data_dir}NEK{nek}/inhib/'
        for k, feat in enumerate(features): 
            print()
            for j, samp in enumerate(samplings): 
                count+=1
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}_df.csv'
                print(f'{count}. NEK{nek} inhib {feat} {samp} {model_type}')
                trainX, trainy, testX, testy = get_arrays(nek_path,file_root)
                train_ds_sizes.append(trainX.shape[0]) 
                train_actives.append(np.sum(trainy==1))
                test_ds_sizes.append(testX.shape[0]) 
                test_actives.append(np.sum(testy==1))
                ds_names.append(file_root)
                neks_label.append(f'NEK{nek}_inhibition')
                samp_types.append(samp)
                feat_types.append(feat)
                # print()
train_df_actives = pd.DataFrame({'NEK': neks_label, 'dataset':ds_names, 'strategy': samp_types,'feat_type':feat_types, 'num_samples':train_ds_sizes, 'actives':train_actives})    
test_df_actives = pd.DataFrame({'NEK': neks_label,'dataset':ds_names, 'strategy': samp_types,'feat_type':feat_types,'num_samples':test_ds_sizes, 'actives':test_actives})    


1. NEK2 bind moe scaled RF
2. NEK2 bind moe UNDER RF
3. NEK2 bind moe SMOTE RF
4. NEK2 bind moe ADASYN RF

5. NEK2 bind mfp scaled RF
6. NEK2 bind mfp UNDER RF
7. NEK2 bind mfp SMOTE RF
8. NEK2 bind mfp ADASYN RF

9. NEK2 inhib moe scaled RF
10. NEK2 inhib moe UNDER RF
11. NEK2 inhib moe SMOTE RF
12. NEK2 inhib moe ADASYN RF

13. NEK2 inhib mfp scaled RF
14. NEK2 inhib mfp UNDER RF
15. NEK2 inhib mfp SMOTE RF
16. NEK2 inhib mfp ADASYN RF

17. NEK3 bind moe scaled RF
18. NEK3 bind moe UNDER RF
19. NEK3 bind moe SMOTE RF
20. NEK3 bind moe ADASYN RF

21. NEK3 bind mfp scaled RF
22. NEK3 bind mfp UNDER RF
23. NEK3 bind mfp SMOTE RF
24. NEK3 bind mfp ADASYN RF

25. NEK5 bind moe scaled RF
26. NEK5 bind moe UNDER RF
27. NEK5 bind moe SMOTE RF
28. NEK5 bind moe ADASYN RF

29. NEK5 bind mfp scaled RF
30. NEK5 bind mfp UNDER RF
31. NEK5 bind mfp SMOTE RF
32. NEK5 bind mfp ADASYN RF

33. NEK9 bind moe scaled RF
34. NEK9 bind moe UNDER RF
35. NEK9 bind moe SMOTE RF
36. NEK9 bind moe ADASYN RF

3

In [78]:
test_df_actives['active%']= (test_df_actives['actives'] / test_df_actives['num_samples'])*100
train_df_actives['active%']= (train_df_actives['actives'] / train_df_actives['num_samples'])*100
total_ds = pd.merge(train_df_actives, test_df_actives, on='dataset', suffixes=('_train', '_test'))
total_ds['NEK'] = total_ds['NEK_train'] 
total_ds['strategy'] = total_ds['strategy_train'] 
total_ds['feat_type'] = total_ds['feat_type_train'] 

total_ds['total_size'] = total_ds['num_samples_train'] + total_ds['num_samples_test']
total_ds['total_actives'] = total_ds['actives_train'] + total_ds['actives_test']
total_ds['total actives/total%'] = (total_ds['total_actives']/ total_ds['total_size'])*100.0
total_ds =total_ds.drop(columns=['NEK_train', 'NEK_test','feat_type_test', 'strategy_test', 'strategy_train', 'feat_type_train' ,])
new_order = ['NEK', 'dataset', 'strategy', 'feat_type', 'num_samples_train', 'actives_train', 'active%_train',
       'num_samples_test', 'actives_test', 'active%_test', 'total_size', 'total_actives', 'total actives/total%'] 
total_ds = total_ds[new_order]

# total_ds = total_ds[(total_ds['strategy'] == 'scaled') & (total_ds['feat_type']=='moe')]
total_ds

total_ds['dataset'] = total_ds['dataset'].str.replace('scaled', 'raw')

total_ds['strategy'] = total_ds['strategy'].str.replace('scaled', 'raw')
# total_ds['NEK'] = total_ds['dataset'].str[0:4]
total_ds.to_csv('all_NEK_dataset_sizes.csv', index=False)
total_ds.columns
size_cols = ['num_samples_train',
       'actives_train', 'active%_train', 'num_samples_test', 'actives_test',
       'active%_test', 'total_size', 'total_actives', 'total actives/total%']

In [69]:
size_cols = ['num_samples_train',
       'actives_train', 'active%_train', 'num_samples_test', 'actives_test',
       'active%_test', 'total_size', 'total_actives', 'total actives/total%']
rf1= pd.read_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/all_NEK_RF_metrics_no_gs.csv')
rf1['strategy']=  rf1['strategy'].str.replace('scaled', 'raw')
nek1 = rf1['NEK'][0]
strategy1= rf1['strategy'][0]
ds_idx = rf1[(rf1['NEK']==nek1) & (rf1['strategy']==strategy1)].index
sizedf_idx = total_ds[(total_ds['NEK']==nek1) & (total_ds['strategy']==strategy1)].loc[[0]]
rf1.loc[ds_idx, size_cols] = sizedf_idx[size_cols].values
rf1[(rf1['NEK']==nek1) & (rf1['strategy']==strategy1)]
# Index(['accuracy', 'precision', 'recall', 'specificity', 'TN', 'FN', 'FP',
#        'TP', 'model', 'cm', 'ROC-AUC', 'MCC', 'Balanced Accuracy', 'f1',
#        'feat_type', 'strategy', 'NEK', 'RF_type'],
#       dtype='object')
# test_df['model'] = test_df['model'].str.replace('scaled', 'raw')

,accuracy,precision,recall,specificity,TN,FN,FP,TP,model,cm,...,RF_type,num_samples_train,actives_train,active%_train,num_samples_test,actives_test,active%_test,total_size,total_actives,total actives/total%
0,0.918728,0.133333,0.166667,0.952030,258,10,13,2,NEK2_binding_moe_scaled_RF,"[258, 13, 10, 2]",...,RF,1125.0,45.0,4.0,283.0,12.0,4.240283,1408.0,57.0,4.048295
1,0.939929,0.142857,0.083333,0.977860,265,11,6,1,NEK2_binding_moe_scaled_RF_BCW,"[265, 6, 11, 1]",...,RF_BCW,1125.0,45.0,4.0,283.0,12.0,4.240283,1408.0,57.0,4.048295
2,0.731449,0.078947,0.500000,0.741697,201,6,70,6,NEK2_binding_moe_scaled_BRFC,"[201, 70, 6, 6]",...,BRFC,1125.0,45.0,4.0,283.0,12.0,4.240283,1408.0,57.0,4.048295
3,0.706714,0.091954,0.666667,0.708487,192,4,79,8,NEK2_binding_moe_scaled_BRFC_BCW,"[192, 79, 4, 8]",...,BRFC_BCW,1125.0,45.0,4.0,283.0,12.0,4.240283,1408.0,57.0,4.048295
16,0.946996,0.333333,0.250000,0.977860,265,9,6,3,NEK2_binding_mfp_scaled_RF,"[265, 6, 9, 3]",...,RF,1125.0,45.0,4.0,283.0,12.0,4.240283,1408.0,57.0,4.048295
17,0.936396,0.333333,0.500000,0.955720,259,6,12,6,NEK2_binding_mfp_scaled_RF_BCW,"[259, 12, 6, 6]",...,RF_BCW,1125.0,45.0,4.0,283.0,12.0,4.240283,1408.0,57.0,4.048295
18,0.823322,0.148148,0.666667,0.830258,225,4,46,8,NEK2_binding_mfp_scaled_BRFC,"[225, 46, 4, 8]",...,BRFC,1125.0,45.0,4.0,283.0,12.0,4.240283,1408.0,57.0,4.048295
19,0.487633,0.076433,1.000000,0.464945,126,0,145,12,NEK2_binding_mfp_scaled_BRFC_BCW,"[126, 145, 0, 12]",...,BRFC_BCW,1125.0,45.0,4.0,283.0,12.0,4.240283,1408.0,57.0,4.048295


In [63]:
binding_df = total_ds[
    (total_ds['NEK'] == 'NEK2_binding') |
    (total_ds['NEK'] == 'NEK3_binding') |
    (total_ds['NEK'] == 'NEK5_binding') |
    (total_ds['NEK'] == 'NEK9_binding')
]
inhibition_df = total_ds[(total_ds['NEK'] == 'NEK2_inhibition') |(total_ds['NEK'] == 'NEK9_inhibition')]
# display(binding_df) 
# display(inhibition_df)

In [ ]:
ratio_df = pd.DataFrame({'dataset':ds_types,'NEK': neknums,'feat_type':moe_mfp, 'strategy':samps, 'train size': train_size, 'train actives': train_actives, 'test size': test_size, 'test actives':test_actives})
ratio_df['dataset size'] = ratio_df['train size']  + ratio_df['test size']
ratio_df = ratio_df[ratio_df['feat_type'] == 'moe']
# ratio_df.sort_values(['NEK', 'strategy'],inplace=True)
ratio_df['total actives'] = ratio_df['train actives']+ratio_df['test actives']
ratio_df['train active/train size'] = ratio_df['train actives'] / ratio_df['train size']*100
ratio_df['test active/test size'] = ratio_df['test actives'] / ratio_df['test size'] *100
ratio_df['train active/total'] = ratio_df['train actives'] / ratio_df['dataset size'] *100
ratio_df['test active/total'] = ratio_df['test actives'] / ratio_df['dataset size'] *100
# plot3 = sns.barplot(data=ratio_df, x='NEK', y='train size')
ratio_df['train inactive per active'] = (ratio_df['train size']-ratio_df['train actives']) / ratio_df['train actives']
ratio_df['test inactive per active'] = (ratio_df['test size']-ratio_df['test actives']) / ratio_df['test actives']
ratio_df.drop(columns='feat_type', inplace=True)
ratio_df.columns
ratio_df


In [84]:
# total_ds.drop(columns='feat_type',inplace=True)
total_ds_onetype = total_ds[total_ds['feat_type'] == 'moe']
total_ds_onetype = total_ds_onetype.drop(columns=['dataset','feat_type'])
total_ds_onetype

,NEK,strategy,num_samples_train,actives_train,active%_train,num_samples_test,actives_test,active%_test,total_size,total_actives,total actives/total%
0,NEK2_binding,raw,1125,45,4.000000,283,12,4.240283,1408,57,4.048295
1,NEK2_binding,UNDER,90,45,50.000000,283,12,4.240283,373,57,15.281501
2,NEK2_binding,SMOTE,2160,1080,50.000000,283,12,4.240283,2443,1092,44.699140
3,NEK2_binding,ADASYN,2158,1078,49.953661,283,12,4.240283,2441,1090,44.653830
8,NEK2_inhibition,raw,1635,112,6.850153,409,28,6.845966,2044,140,6.849315
9,NEK2_inhibition,UNDER,224,112,50.000000,409,28,6.845966,633,140,22.116904
10,NEK2_inhibition,SMOTE,3046,1523,50.000000,409,28,6.845966,3455,1551,44.891462
11,NEK2_inhibition,ADASYN,3037,1514,49.851827,409,28,6.845966,3446,1542,44.747533
16,NEK3_binding,raw,1122,64,5.704100,282,17,6.028369,1404,81,5.769231
17,NEK3_binding,UNDER,128,64,50.000000,282,17,6.028369,410,81,19.756098


In [82]:
total_ds_onetype

,NEK,strategy,feat_type,num_samples_train,actives_train,active%_train,num_samples_test,actives_test,active%_test,total_size,total_actives,total actives/total%
0,NEK2_binding,raw,moe,1125,45,4.000000,283,12,4.240283,1408,57,4.048295
1,NEK2_binding,UNDER,moe,90,45,50.000000,283,12,4.240283,373,57,15.281501
2,NEK2_binding,SMOTE,moe,2160,1080,50.000000,283,12,4.240283,2443,1092,44.699140
3,NEK2_binding,ADASYN,moe,2158,1078,49.953661,283,12,4.240283,2441,1090,44.653830
8,NEK2_inhibition,raw,moe,1635,112,6.850153,409,28,6.845966,2044,140,6.849315
9,NEK2_inhibition,UNDER,moe,224,112,50.000000,409,28,6.845966,633,140,22.116904
10,NEK2_inhibition,SMOTE,moe,3046,1523,50.000000,409,28,6.845966,3455,1551,44.891462
11,NEK2_inhibition,ADASYN,moe,3037,1514,49.851827,409,28,6.845966,3446,1542,44.747533
16,NEK3_binding,raw,moe,1122,64,5.704100,282,17,6.028369,1404,81,5.769231
17,NEK3_binding,UNDER,moe,128,64,50.000000,282,17,6.028369,410,81,19.756098
